<a href="https://colab.research.google.com/github/kotagiri-kulbhushan/weather-crop-advisory/blob/main/weather_and_crop_advisory__.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain langchain-community langchain-openai faiss-cpu requests python-dotenv scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
import os
import requests
import pandas as pd
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.llms import HuggingFaceHub

# OpenWeather API Key
OPENWEATHER_API_KEY = "1ee3915e4b685fb8eef0ee746a2d470e"


In [3]:
def get_weather(city):
    """
    Fetch real-time weather from OpenWeatherMap
    """
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHER_API_KEY}&units=metric"

    try:
        response = requests.get(url, timeout=10)
        data = response.json()

        if data.get("cod") != 200:
            print(f" Weather API Error: {data.get('message', 'Unknown error')}")
            # Return mock data if API fails
            return {
                "temperature": 28,
                "humidity": 60,
                "pressure": 1010,
                "rain": 0,
                "description": "clear sky (mock data)"
            }

        weather = {
            "temperature": data["main"]["temp"],
            "humidity": data["main"]["humidity"],
            "pressure": data["main"]["pressure"],
            "rain": data.get("rain", {}).get("1h", 0),
            "description": data["weather"][0]["description"]
        }
        return weather
    except Exception as e:
        print(f" Error fetching weather: {e}")
        # Return mock data
        return {
            "temperature": 28,
            "humidity": 60,
            "pressure": 1010,
            "rain": 0,
            "description": "clear sky (mock data)"
        }


In [4]:
agri_text = """
Crop Recommendations:

1. Rice: Grows best in areas with high rainfall (200-300mm) and clayey soil.
   Ideal temperature: 20-30°C. Requires high humidity (70-80%).

2. Wheat: Requires moderate temperature (15-25°C) and well-drained loamy soil.
   Prefers low to moderate humidity (50-70%). Needs cool weather for germination.

3. Maize: Prefers warm weather (20-30°C) and moderate rainfall (50-100mm).
   Grows well in well-drained soil. Requires moderate humidity (60-70%).

4. Cotton: Grows in black soil and requires dry weather (less than 50mm rain).
   Ideal temperature: 25-35°C. Needs low humidity (40-60%) and plenty of sunshine.

5. Sugarcane: Needs high rainfall (150-250mm) and fertile soil.
   Ideal temperature: 25-35°C. Requires high humidity (70-85%).

6. Millets: Suitable for dry, less fertile soil. Very drought-resistant.
   Ideal temperature: 25-35°C. Can grow with minimal rainfall (30-50mm).

7. Pulses: Prefer dry and light soil. Require moderate temperature (20-30°C).
   Low humidity (40-60%) and minimal rainfall (40-80mm).

8. Groundnut: Needs sandy soil and moderate rainfall (50-100mm).
   Ideal temperature: 25-30°C. Prefers moderate humidity (60-70%).

9. Soybean: Grows well in warm weather (25-30°C) with moderate rainfall (100-150mm).
   Requires well-drained loamy soil and moderate humidity (60-75%).

10. Barley: Cold-tolerant crop, grows best at 15-20°C.
    Requires minimal rainfall (40-80mm) and low humidity (40-60%).
"""


In [5]:
# Setting up RAG system

# Split text into chunks
text_splitter = CharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    separator="\n"
)

texts = text_splitter.split_text(agri_text)
docs = [Document(page_content=t) for t in texts]

print(f" Created {len(docs)} document chunks")

# Create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Creating vector database

# Create FAISS vector store
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

#  RAG system ready!"



 Created 9 document chunks


/tmp/ipython-input-72048010.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
def get_simple_recommendation(weather):
    """
    Simple logic-based recommendation (no LLM needed)
    """
    temp = weather['temperature']
    humidity = weather['humidity']
    rain = weather['rain']

    recommendations = []

    # Temperature-based suggestions
    if temp < 20:
        recommendations.append("Wheat (cool weather crop)")
        recommendations.append("Barley (cold-tolerant)")
    elif 20 <= temp <= 30:
        recommendations.append("Rice (moderate temperature)")
        recommendations.append("Maize (warm weather)")
        recommendations.append("Soybean (moderate warmth)")
    else:  # temp > 30
        recommendations.append("Cotton (hot weather)")
        recommendations.append("Sugarcane (high temperature)")
        recommendations.append("Millets (drought-resistant)")

    # Humidity-based suggestions
    if humidity > 70:
        if "Rice" not in str(recommendations):
            recommendations.append("Rice (high humidity)")
    elif humidity < 50:
        if "Cotton" not in str(recommendations):
            recommendations.append("Cotton (dry conditions)")

    # Rainfall-based suggestions
    if rain > 10:
        recommendations.append("Sugarcane (high rainfall)")
    elif rain < 2:
        recommendations.append("Millets (low water requirement)")

    return recommendations[:3]


In [7]:
def get_rag_context(weather):
    """
    Retrieve relevant crop information from knowledge base
    """
    query = f"temperature {weather['temperature']}°C humidity {weather['humidity']}% rain {weather['rain']}mm"
    relevant_docs = retriever.invoke(query)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])
    return context


In [8]:
def get_crop_advice(city):
    """
    Main function to get crop recommendations
    """
    print("\n" + "="*40)
    print(f"🌾 CROP ADVISORY FOR: {city.upper()}")
    print("="*40)

    # Get weather
    weather = get_weather(city)

    print("\n🌦️  CURRENT WEATHER:")
    print(f"   Temperature: {weather['temperature']}°C")
    print(f"   Humidity: {weather['humidity']}%")
    print(f"   Pressure: {weather['pressure']} hPa")
    print(f"   Rain: {weather['rain']} mm")
    print(f"   Description: {weather['description']}")

    # Get simple recommendations
    simple_recs = get_simple_recommendation(weather)

    print("\n🌾 TOP CROP RECOMMENDATIONS:")
    for i, crop in enumerate(simple_recs, 1):
        print(f"   {i}. {crop}")

    # Get RAG context
    print("\n📚 KNOWLEDGE BASE INSIGHTS:")
    rag_context = get_rag_context(weather)
    lines = rag_context.split('\n')
    relevant_info = [line for line in lines if line.strip() and not line.startswith('Crop Recommendations')]
    for info in relevant_info[:3]:
        if info.strip():
            print(f"   • {info.strip()}")

    # Generate advice
    print("\n💡 FARMING ADVICE:")
    temp = weather['temperature']
    humidity = weather['humidity']
    rain = weather['rain']

    if temp > 30:
        print("   ⚠️  High temperature - ensure adequate irrigation")
    elif temp < 20:
        print("   ❄️  Cool weather - good for winter crops")
    else:
        print("   ✅ Moderate temperature - ideal for most crops")

    if humidity > 70:
        print("   💧 High humidity - monitor for fungal diseases")
    elif humidity < 50:
        print("   ☀️  Low humidity - increase irrigation frequency")
    else:
        print("   ✅ Moderate humidity - optimal conditions")

    if rain > 5:
        print("   🌧️  Recent rainfall - reduce irrigation, check drainage")
    elif rain == 0:
        print("   ☀️  No recent rain - ensure regular watering")

    print("\n" + "="*60)

    return {
        "city": city,
        "weather": weather,
        "recommendations": simple_recs,
        "rag_context": rag_context
    }


In [9]:


# Test with Delhi
result = get_crop_advice("Delhi")
result = get_crop_advice("bangalore")



🌾 CROP ADVISORY FOR: DELHI

🌦️  CURRENT WEATHER:
   Temperature: 14.05°C
   Humidity: 67%
   Pressure: 1019 hPa
   Rain: 0 mm
   Description: haze

🌾 TOP CROP RECOMMENDATIONS:
   1. Wheat (cool weather crop)
   2. Barley (cold-tolerant)
   3. Millets (low water requirement)

📚 KNOWLEDGE BASE INSIGHTS:
   • Low humidity (40-60%) and minimal rainfall (40-80mm).
   • 8. Groundnut: Needs sandy soil and moderate rainfall (50-100mm).
   • Ideal temperature: 25-30°C. Prefers moderate humidity (60-70%).

💡 FARMING ADVICE:
   ❄️  Cool weather - good for winter crops
   ✅ Moderate humidity - optimal conditions
   ☀️  No recent rain - ensure regular watering


🌾 CROP ADVISORY FOR: BANGALORE

🌦️  CURRENT WEATHER:
   Temperature: 21.55°C
   Humidity: 60%
   Pressure: 1015 hPa
   Rain: 0 mm
   Description: scattered clouds

🌾 TOP CROP RECOMMENDATIONS:
   1. Rice (moderate temperature)
   2. Maize (warm weather)
   3. Soybean (moderate warmth)

📚 KNOWLEDGE BASE INSIGHTS:
   • Ideal temperature: 25-3